In [99]:
import pandas as pd

In [100]:
# our dataframe
csv_file = '../data/listings.csv'
df = pd.read_csv(csv_file) # get data in form of dataframe

### Location table

In [101]:
location=df.loc[:,[
    'neighbourhood',
    'latitude',
    'longitude',
    'neighborhood_overview',
]]

In [102]:
location.rename(columns={
    'neighbourhood' : 'neighborhood'
}, inplace=True)

In [103]:
location.head()

,neighborhood,latitude,longitude,neighborhood_overview
0,"Chicago, Illinois, United States",41.787900,-87.587800,The apartment is less than one block from beau...
1,"Chicago, Illinois, United States",41.902890,-87.681820,"Ukrainian Village was just named ""Hottest Neig..."
2,NaN,41.911830,-87.640000,NaN
3,"Chicago, Illinois, United States",41.923350,-87.649510,The Guest House is in the heart of Chicago's m...
4,"Chicago, Illinois, United States",41.925961,-87.656364,Lincoln Park is a great neighborhood where peo...


### Host table

In [104]:
host=df.loc[:,[
    'host_id',
    'host_url',
    'host_name',
    'host_since',
    'host_location',
    'host_about',
    'host_response_rate',
    'host_acceptance_rate',
    'host_is_superhost',
    'host_neighbourhood',
    'host_listings_count',
    'host_total_listings_count',
    'host_verifications', # -> email, phone
    # 'email_verified',
    # 'phone_verified',
    'host_has_profile_pic',
    'host_identity_verified',
]]

In [105]:
# remove 'host_' prefix from col names
for col_i in range(1,len(host.columns)):
	if 'host_' in host.columns[col_i]:
		new = host.columns[col_i].replace('host_','')
		host.rename(columns={
			host.columns[col_i] : new
        }, inplace=True)

In [106]:
# create lists of bools for verified email / phone
email_bools, phone_bools = [], []
for v in host['verifications']:
	email_bools.append(True) if "email" in v else email_bools.append(False)
	phone_bools.append(True) if "phone" in v else phone_bools.append(False)

# insert columns in host df
host.insert(14, 'email_verified', email_bools) # host['email_verified'] = email_bools
host.insert(15, 'phone_verified', phone_bools) # host['phone_verified'] = phone_bools

# delete verifications column
del host['verifications']

In [107]:
# change british spelling
host.rename(columns={
	'neighbourhood' : 'neighborhood'
}, inplace=True)

In [ ]:
i = 0
for val in host.response_rate:
    if isinstance(val, str):
        host.response_rate[i] = float(val.strip('%'))
    i += 1

j = 0
for val in host.acceptance_rate:
    if isinstance(val, str):
        host.acceptance_rate[j] = float(val.strip('%'))
    j += 1

In [123]:
host.head()

,host_id,url,name,since,location,about,response_rate,acceptance_rate,is_superhost,neighborhood,listings_count,total_listings_count,has_profile_pic,email_verified,phone_verified,identity_verified
0,2613,https://www.airbnb.com/users/show/2613,Rebecca,2008-08-29,"Chicago, IL",My 2 bdrm apartment is a 2nd floor walk-up in ...,100.0,97.0,t,Hyde Park,1,1,t,True,True,t
1,17928,https://www.airbnb.com/users/show/17928,Sarah,2009-05-19,"Chicago, IL",We live in Chicago. We love to travel and see ...,100.0,94.0,t,Ukrainian Village,2,2,t,True,True,t
2,33004,https://www.airbnb.com/users/show/33004,At Home Inn,2009-08-21,"Chicago, IL","Hi, we're Bob and Liz Biddle, long time Chicag...",100.0,99.0,t,Old Town,10,82,t,True,True,t
3,46734,https://www.airbnb.com/users/show/46734,Sharon And Robert,2009-10-18,"Chicago, IL",Best in the city with long-time urban-dwelling...,100.0,89.0,f,Lincoln Park,1,1,t,True,True,t
4,101521,https://www.airbnb.com/users/show/101521,Red,2010-03-31,"Chicago, IL",Travel is fun and exciting but can be hard on ...,95.0,71.0,f,Lincoln Park,8,12,t,True,True,t


### Reviews table

In [110]:
reviews=df.loc[:,[
	'number_of_reviews',
    'first_review',
    'last_review',
    'reviews_per_month',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]]

In [111]:
import re
for col in reviews.columns:
    if "review_scores" in col:
        new = re.sub("review_scores_", "", col) + "_score"
        reviews.rename(columns = {
            col : new
        }, inplace=True)

In [112]:
reviews.head()

,number_of_reviews,first_review,last_review,reviews_per_month,rating_score,accuracy_score,cleanliness_score,checkin_score,communication_score,location_score,value_score
0,211,2015-01-09,2022-11-18,2.18,4.99,4.98,4.99,4.99,4.99,4.96,4.93
1,475,2009-07-03,2022-12-05,2.90,4.69,4.85,4.56,4.91,4.87,4.88,4.75
2,59,2014-04-28,2022-11-26,0.56,4.64,4.75,4.80,4.80,4.78,4.98,4.64
3,13,2015-06-29,2022-09-12,0.14,5.00,5.00,5.00,5.00,5.00,5.00,4.85
4,41,2010-05-24,2022-11-12,0.27,4.27,4.03,3.97,4.10,4.00,4.90,4.17


### Listing table

In [113]:
listing=df.loc[:,[
    'listing_url',
    'name',
    'description',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'price',
    'minimum_nights',
    'maximum_nights',
]]

In [114]:
max(listing.price)

'$999.00'

In [ ]:
i = 0
for val in listing.price:
    if isinstance(val, str):
        if '$' in val:
            val = val.strip('$')
        if ',' in val:
            val = val.replace(',','')
        listing.price[i] = float(val)
    i += 1

In [116]:
listing.head()

,listing_url,name,description,property_type,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights
0,https://www.airbnb.com/rooms/2384,Hyde Park - Walk to UChicago,You are invited to be the sole Airbnb guest in...,Private room in condo,Private room,1,1 shared bath,1.0,1.0,"[""Wine glasses"", ""Smoke alarm"", ""Paid parking ...",84.0,3,89
1,https://www.airbnb.com/rooms/7126,Tiny Studio Apartment 94 Walk Score,A very small studio in a wonderful neighborhoo...,Entire rental unit,Entire home/apt,2,1 bath,1.0,1.0,"[""Wine glasses"", ""Smoke alarm"", ""Iron"", ""Oven""...",85.0,2,60
2,https://www.airbnb.com/rooms/10945,The Biddle House (#1),Beautiful first floor apartment in Historic Ol...,Entire rental unit,Entire home/apt,4,1 bath,2.0,2.0,"[""Smoke alarm"", ""Iron"", ""Oven"", ""Long term sta...",95.0,4,1125
3,https://www.airbnb.com/rooms/12140,Lincoln Park Guest House,Bed and Breakfast license issued by the City o...,Room in boutique hotel,Private room,3,1 private bath,1.0,1.0,"[""Smoke alarm"", ""Iron"", ""First aid kit"", ""Fire...",329.0,2,7
4,https://www.airbnb.com/rooms/24833,Prime LincolnPark 1 Block Fullerton Express L ...,Perfect private entrance apartment one block f...,Entire rental unit,Entire home/apt,3,1 bath,1.0,1.0,"[""Wine glasses"", ""Babysitter recommendations"",...",64.0,32,395


## Insert records to SQL database

In [117]:
import psycopg2
from sqlalchemy import create_engine

In [118]:
conn_string = 'postgresql+psycopg2://julieannscherer@localhost:5432/airbnb'
engine = create_engine(conn_string)
# conn = engine.connect()

In [119]:
location.to_sql(
	name='location', 
  con=engine, 
  if_exists='append',
  index=False,
)

625

In [120]:
host.to_sql(
  name='host', 
  con=engine, 
  if_exists='append',
  index=False,
)

625

In [121]:
reviews.to_sql(
  name='reviews', 
  con=engine, 
  if_exists='append',
  index=False,
)

625

In [122]:
listing.to_sql(
  name='listing', 
  con=engine, 
  if_exists='append',
  index=False,
)

625